# Generator
This notebook generates two types of equations:
- one-line inequalities (incl. powers)
- equalities with a fraction
It generates train and test image

In [2]:
!pip3 install tensorflow

    100% |████████████████████████████████| 83.1MB 24kB/s  eta 0:00:01
    100% |████████████████████████████████| 1.1MB 1.5MB/s eta 0:00:01
    100% |████████████████████████████████| 102kB 6.8MB/s ta 0:00:01
    100% |████████████████████████████████| 3.1MB 597kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 61kB 6.5MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/ad/15/690c13ae714e156491392cdbdbf41b485d23c285aa698239a67f7cfc9e0a/numpy-1.16.1-cp35-cp35m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 10.6MB 193kB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 5.8MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d1/6a/4b2fcefd2ea0868810e92d519dacac1ddc64a2e53ba9e3422c3b62b378a6/setuptools-40.8.0-py2.py3-none-any.whl
    100% |██████████████████

In [49]:
import numpy as np
from os import listdir, path, mkdir
from skimage import io
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.transform import resize, rotate
from skimage.util import random_noise, invert
from skimage.color import gray2rgb

import tensorflow as tf
import math
import json
import cv2

In [50]:
read_path = "extracted_images"
train = False
if not path.exists('normalized'):
    mkdir('normalized')
if not path.exists("formulas/train"):
    mkdir("formulas/train")
if not path.exists("formulas/test"):
    mkdir("formulas/test")
if train:
    write_single_path = "normalized/train"
    formula_path = "formulas/train/fractions"
else:
    write_single_path = "normalized/test"
    formula_path = "formulas/test/fractions"

In [51]:
label_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-', '+', '=', 'leq', 'neq', 'geq', 'alpha',
               'beta', 'lambda', 'lt', 'gt', 'x', 'y']
label_names_dict = dict()
for label,no in zip(label_names,list(range(len(label_names)))):
    label_names_dict[label] = no
    
    if not path.exists(write_single_path):
        mkdir(write_single_path)
    if not path.exists(write_single_path+"/"+label):
        mkdir(write_single_path+"/"+label)
print(label_names_dict)

{'8': 8, 'geq': 15, 'gt': 20, '-': 10, 'y': 22, '3': 3, '9': 9, '0': 0, 'lambda': 18, 'alpha': 16, '6': 6, '7': 7, '5': 5, '=': 12, '+': 11, '2': 2, 'beta': 17, 'leq': 13, '4': 4, 'neq': 14, 'lt': 19, 'x': 21, '1': 1}


In [52]:
def crop(img):
    crop = np.copy(img)/255
    h,w = img.shape
    left = 0
    while left < w//2 and np.sum(crop[:,left]) >= 0.98*h:
        left += 1
    right = w-1
    while right > w//2 and np.sum(crop[:,right]) >= 0.98*h:
        right -= 1
    if left > 0:
        left -1
    if right < h-1:
        right += 1
    crop = crop[:,left:right]
    
    top = 0
    while top < h//2 and np.sum(crop[top,:]) >= 0.98*w:
        top += 1
    bottom = h-1
    while bottom > h//2 and np.sum(crop[bottom,:]) >= 0.98*w:
        bottom -= 1
    if top > 0:
        top -= 1
    if bottom < h-1:
        bottom += 1
    crop = crop[top:bottom,:]*255
    return crop

def add_symbol_to_image(img,folder,choices,padding,minsize,maxsize,bpower=False,bsmall=False,bnom=False,bden=False,
                        width=False):
    choice = np.random.randint(len(choices))
    symbol_img = io.imread(read_path+"/"+folder+"/"+choices[choice])
    new_width = np.random.randint(minsize,maxsize+1)
    new_height = np.random.randint(minsize,maxsize+1)
    if width is not False:
        new_width = width
    symbol_img_res = resize(symbol_img, (new_height, new_width), cval=1)*255
    symbol_img_res = crop(symbol_img_res)
    new_height, new_width = symbol_img_res.shape
    shift = np.random.randint(-4+(60-new_height)//2,4+(60-new_height)//2)

    
    bounding_box = {
        'xmin': padding,
        'xmax': padding+new_width,
        'ymin': 65+shift-15*bpower+10*bsmall-30*bnom+30*bden,
        'ymax': 65+shift+new_height-15*bpower+10*bsmall-30*bnom+30*bden,
        'class_text': folder,
        'class': label_names_dict[folder]
    }
    
    if folder == "y" or folder == "beta":
        bounding_box['ymin'] += 10
        bounding_box['ymax'] += 10
                       
    
    xmin, xmax = bounding_box['xmin'],bounding_box['xmax']
    ymin, ymax = bounding_box['ymin'],bounding_box['ymax']
    
    img[ymin:ymax,xmin:xmax] += invert(symbol_img_res)+254
    padding += new_width+np.random.randint(2,5)
    
    return img,padding,bounding_box

def add_rectangles(img, bounding_boxes):
    img_color = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    for bounding_box in bounding_boxes[1:]:
        xmin, xmax = bounding_box['xmin'], bounding_box['xmax']
        ymin, ymax = bounding_box['ymin'], bounding_box['ymax']
        img_color[ymin,xmin:xmax] = [255,0,0]
        img_color[ymax,xmin:xmax] = [255,0,0]
        img_color[ymin:ymax,xmin] = [255,0,0]
        img_color[ymin:ymax,xmax] = [255,0,0]
    return img_color

In [53]:
def normalize_single(symbol):
    symbol = np.copy(symbol)
    symbol /= np.max(symbol)
    rows, cols = symbol.shape
    # scale to 40x40
    inner_size = 40
    if rows > cols:
        factor = inner_size/rows
        rows = inner_size
        cols = int(round(cols*factor))
        inner = cv2.resize(symbol, (cols,rows))
    else:
        factor = inner_size/cols
        cols = inner_size
        rows = int(round(rows*factor))
        inner = cv2.resize(symbol, (cols, rows))
        
    # pad to 48x48
    outer_size = 48
    colsPadding = (int(math.ceil((outer_size-cols)/2.0)),int(math.floor((outer_size-cols)/2.0)))
    rowsPadding = (int(math.ceil((outer_size-rows)/2.0)),int(math.floor((outer_size-rows)/2.0)))
    outer = np.pad(inner,(rowsPadding,colsPadding),'constant', constant_values=(1,1))
    
    # center the mass
    shiftx,shifty = getBestShift(outer)
    shifted = shift(outer,shiftx,shifty)
    return shifted
    
def getBestShift(img):
    inv = invert(img)
    cy,cx = ndimage.measurements.center_of_mass(inv)

    rows,cols = img.shape
    shiftx = np.round(cols/2.0-cx).astype(int)
    shifty = np.round(rows/2.0-cy).astype(int)

    return shiftx,shifty

def shift(img,sx,sy):
    rows,cols = img.shape
    M = np.float32([[1,0,sx],[0,1,sy]])
    shifted = cv2.warpAffine(img,M,(cols,rows), borderValue=1)
    return shifted  

In [54]:
# simple equations
list_digits = []
for i in range(10):
    list_digits.append(listdir(read_path+"/"+str(i)))
list_plus = listdir(read_path+"/+")
list_minus = listdir(read_path+"/-")
list_lt = listdir(read_path+"/lt")
list_gt= listdir(read_path+"/gt")
list_leq = listdir(read_path+"/leq")
list_geq = listdir(read_path+"/geq")
list_neq = listdir(read_path+"/neq")
list_equal = listdir(read_path+"/=")

list_alpha = listdir(read_path+"/alpha")
list_beta = listdir(read_path+"/beta")
list_lambda = listdir(read_path+"/lambda")
list_x = listdir(read_path+"/x")
list_y = listdir(read_path+"/y")

if train: 
    for i in range(10):
        list_digits[i] = list_digits[i][len(list_digits[i])//4:]
    list_plus = list_plus[len(list_plus)//4:]
    list_minus = list_minus[len(list_minus)//4:]
    list_lt = list_lt[len(list_lt)//4:]
    list_gt = list_gt[len(list_gt)//4:]
    list_leq = list_leq[len(list_leq)//4:]
    list_geq = list_geq[len(list_geq)//4:]
    list_neq = list_neq[len(list_neq)//4:]
    list_equal = list_equal[len(list_equal)//4:]
    list_alpha = list_alpha[len(list_alpha)//4:]
    list_beta = list_beta[len(list_beta)//4:]
    list_lambda = list_lambda[len(list_lambda)//4:]
    list_x = list_x[len(list_x)//4:]
    list_y = list_y[len(list_y)//4:]
else:
    for i in range(10):
        list_digits[i] = list_digits[i][:len(list_digits[i])//4]
    list_plus = list_plus[:len(list_plus)//4]
    list_minus = list_minus[:len(list_minus)//4]
    list_lt = list_lt[:len(list_lt)//4]
    list_gt = list_gt[:len(list_gt)//4]
    list_leq = list_leq[:len(list_leq)//4]
    list_geq = list_geq[:len(list_geq)//4]
    list_neq = list_neq[:len(list_neq)//4]
    list_equal = list_equal[:len(list_equal)//4]
    list_alpha = list_alpha[:len(list_alpha)//4]
    list_beta = list_beta[:len(list_beta)//4]
    list_lambda = list_lambda[:len(list_lambda)//4]
    list_x = list_x[:len(list_x)//4]
    list_y = list_y[:len(list_y)//4]

list_mid = [list_minus,list_plus]
list_end = [list_lt,list_gt,list_leq,list_geq,list_neq,list_equal]  
list_variables = [list_alpha,list_beta,list_lambda,list_x,list_y]
var_names = ["#alpha","#beta","#lambda","x","y"]
    
bounding_boxes = []
for i in range(5000):
    random_name = str(np.random.randint(1,99999))
    img = np.zeros((140,60*(4+4+4)))
    rand_num_1 = np.random.randint(1,1000)
    rand_num_2 = np.random.randint(1,1000)
    mid = np.random.randint(2)
    if mid:
        mid_str = "+"
        result = rand_num_1+rand_num_2
    else:
        mid_str = "-"
        result = rand_num_1-rand_num_2
    result_type = np.random.randint(6)
    if result_type == 0:
        end_str = " #lt "
        result += np.random.randint(100)
    elif result_type == 1:
        end_str = " #gt "
        result -= np.random.randint(100)
    elif result_type == 2:
        end_str = " #leq "
        result += np.random.randint(100)
    elif result_type == 3:
        end_str = " #geq "
        result -= np.random.randint(100)
    elif result_type == 4:
        end_str = " #neq "
        result += 1+np.random.randint(100)
    else:
        end_str = " = "
    if end_str != " = ":
        var_type = np.random.randint(len(list_variables))
        var = var_names[var_type]
        exp = "^"+str(np.random.randint(2,5))
    else:
        var = ""
        exp = ""
    
    rand_num_1_str = str(rand_num_1)
    rand_num_2_str = str(rand_num_2)
    result_str = str(result)
    num_strs = [rand_num_1_str,rand_num_2_str,result_str]
    filename = rand_num_1_str+var+exp+mid_str+rand_num_2_str+end_str+result_str+"_"+random_name+".jpg"
    print("Filename: ", filename)
    bounding_box = [{'filename': filename}]
    padding = 5
    class_names = []
    for k in range(3):
        for j in range(len(num_strs[k])):
            if j == 0 and num_strs[k][0] == '-':
                img, padding, new_bounding_box = add_symbol_to_image(img,'-',list_minus,padding,39,45)
                bounding_box.append(new_bounding_box)
                class_names.append('-')
            else:
                digit = int(num_strs[k][j])
                img, padding, new_bounding_box = add_symbol_to_image(img,str(digit),list_digits[digit],
                                                                     padding,55,60)
                bounding_box.append(new_bounding_box)
                class_names.append(str(digit))
        if k == 0:
            if var != "":
                var_str_crop = var.strip()
                var_str_crop = var_str_crop.replace("#","")
                img, padding, new_bounding_box = add_symbol_to_image(img,var_str_crop,list_variables[var_type],
                                                                     padding,40,45,bsmall=True)
                bounding_box.append(new_bounding_box)
                class_names.append(var_str_crop)
                
                pdigit = int(exp[1:])
                img, padding, new_bounding_box = add_symbol_to_image(img,str(pdigit),list_digits[pdigit],
                                                                     padding,35,40,bpower=True)
                bounding_box.append(new_bounding_box)
                class_names.append(exp[1:])
                
            img, padding, new_bounding_box = add_symbol_to_image(img,mid_str,list_mid[mid],
                                                                     padding,39,54)
            bounding_box.append(new_bounding_box)
            class_names.append(mid_str)
        elif k == 1:
            end_str_crop = end_str.strip()
            end_str_crop = end_str_crop.replace("#","")
            img, padding, new_bounding_box = add_symbol_to_image(img,end_str_crop,list_end[result_type],
                                                                     padding,39,54)
            bounding_box.append(new_bounding_box)
            class_names.append(end_str_crop)
            
    bounding_boxes.append(bounding_box)
    img = invert(img)+254
#     plt.figure(figsize=(20,10))
#     plt.imshow(img, cmap="gray")
#     plt.show()
    for bb,cname in zip(bounding_box[1:],class_names):
        xmin, xmax = bb['xmin'], bb['xmax']
        ymin, ymax = bb['ymin'], bb['ymax']
    
        normed = normalize_single(img[ymin:ymax+1,xmin:xmax+1])
        for xi in range(len(normed)):
            for xj in range(len(normed[0])):
                if normed[xi][xj] > 1:
                    normed[xi][xj] = 1.0
                    
        r = np.random.randint(9999)
        io.imsave(write_single_path+"/"+cname+"/"+cname+"_"+str(r)+".jpg", normed)
    
    if not path.exists(formula_path):
        mkdir(formula_path)
    io.imsave(formula_path+"/"+filename, img/255)
    print("Finished: ", i)


Filename:  974#lambda^2+468 #leq 1487_51538.jpg
Finished:  0
Filename:  417x^3-98 #geq 291_1493.jpg
Finished:  1
Filename:  305-242 = 63_29396.jpg
Finished:  2
Filename:  741#alpha^3-521 #geq 127_70069.jpg
Finished:  3
Filename:  928-690 = 238_80031.jpg
Finished:  4
Filename:  545y^3+751 #lt 1377_33088.jpg
Finished:  5
Filename:  567#beta^2+470 #leq 1054_86162.jpg
Finished:  6
Filename:  76y^3-630 #gt -592_51833.jpg
Finished:  7
Filename:  725#alpha^2-550 #lt 199_82492.jpg
Finished:  8
Filename:  586-125 = 461_18336.jpg
Finished:  9
Filename:  544-432 = 112_38850.jpg
Finished:  10
Filename:  161+719 = 880_82714.jpg
Finished:  11
Filename:  936+219 = 1155_4452.jpg
Finished:  12
Filename:  890#alpha^4+519 #leq 1498_91933.jpg
Finished:  13
Filename:  682y^4-136 #gt 511_47243.jpg
Finished:  14
Filename:  243#beta^4-977 #geq -811_15949.jpg
Finished:  15
Filename:  272x^4-53 #neq 311_965.jpg
Finished:  16
Filename:  435y^2-656 #neq -160_77155.jpg
Finished:  17
Filename:  128-982 = -854_621.j

Finished:  146
Filename:  784x^2-889 #leq -54_28384.jpg
Finished:  147
Filename:  343#beta^2-875 #leq -489_79315.jpg
Finished:  148
Filename:  600#alpha^2-709 #lt -27_79227.jpg
Finished:  149
Filename:  744#alpha^3-74 #neq 728_48005.jpg
Finished:  150
Filename:  647x^2-728 #geq -153_34539.jpg
Finished:  151
Filename:  916-536 = 380_90976.jpg
Finished:  152
Filename:  609-557 = 52_68213.jpg
Finished:  153
Filename:  27y^4+398 #neq 481_49773.jpg
Finished:  154
Filename:  820#beta^2-670 #geq 65_31409.jpg
Finished:  155
Filename:  241x^2-655 #geq -441_95146.jpg
Finished:  156
Filename:  111y^3+843 #geq 928_16809.jpg
Finished:  157
Filename:  866-229 = 637_20853.jpg
Finished:  158
Filename:  633#alpha^2-554 #geq -13_85230.jpg
Finished:  159
Filename:  452#beta^3-615 #neq -134_13746.jpg
Finished:  160
Filename:  949-110 = 839_87903.jpg
Finished:  161
Filename:  426+385 = 811_78036.jpg
Finished:  162
Filename:  718y^4+590 #gt 1224_82203.jpg
Finished:  163
Filename:  687+113 = 800_27486.jpg
Fi

Finished:  291
Filename:  276#beta^3+567 #geq 751_21869.jpg
Finished:  292
Filename:  195#lambda^3-458 #lt -247_52564.jpg
Finished:  293
Filename:  624#alpha^4-980 #leq -283_34393.jpg
Finished:  294
Filename:  582-594 = -12_79908.jpg
Finished:  295
Filename:  581-702 = -121_18235.jpg
Finished:  296
Filename:  883#beta^4+375 #lt 1339_84134.jpg
Finished:  297
Filename:  609#beta^2-870 #gt -284_45026.jpg
Finished:  298
Filename:  484#alpha^3+161 #leq 681_2545.jpg
Finished:  299
Filename:  979#beta^4+750 #lt 1824_42865.jpg
Finished:  300
Filename:  441#beta^4-180 #leq 354_56974.jpg
Finished:  301
Filename:  520y^3-667 #leq -70_50489.jpg
Finished:  302
Filename:  954#lambda^4-232 #neq 814_21675.jpg
Finished:  303
Filename:  796y^4+511 #geq 1263_57458.jpg
Finished:  304
Filename:  50#lambda^3+637 #leq 756_48773.jpg
Finished:  305
Filename:  286+793 = 1079_2892.jpg
Finished:  306
Filename:  396#alpha^3+708 #lt 1140_95064.jpg
Finished:  307
Filename:  107#beta^4-993 #lt -810_10636.jpg
Finished

Finished:  436
Filename:  74-315 = -241_73347.jpg
Finished:  437
Filename:  614#alpha^2+592 #leq 1209_21181.jpg
Finished:  438
Filename:  623x^3-871 #gt -281_93164.jpg
Finished:  439
Filename:  141#lambda^2+335 #lt 509_20456.jpg
Finished:  440
Filename:  100-14 = 86_38789.jpg
Finished:  441
Filename:  75-342 = -267_53036.jpg
Finished:  442
Filename:  656y^3-937 #neq -255_7055.jpg
Finished:  443
Filename:  468-441 = 27_79135.jpg
Finished:  444
Filename:  795#beta^2-248 #gt 484_83477.jpg
Finished:  445
Filename:  177-365 = -188_19482.jpg
Finished:  446
Filename:  284y^3-741 #gt -458_68293.jpg
Finished:  447
Filename:  152#beta^3+894 #geq 966_46192.jpg
Finished:  448
Filename:  802x^4+658 #geq 1435_52024.jpg
Finished:  449
Filename:  83x^3-307 #leq -176_12878.jpg
Finished:  450
Filename:  691+523 = 1214_45962.jpg
Finished:  451
Filename:  84#lambda^4-667 #leq -487_69456.jpg
Finished:  452
Filename:  209#lambda^4+482 #gt 673_34590.jpg
Finished:  453
Filename:  963-906 = 57_14713.jpg
Finish

Finished:  579
Filename:  512#beta^4-543 #neq -24_16945.jpg
Finished:  580
Filename:  892y^3+945 #gt 1806_44862.jpg
Finished:  581
Filename:  77#alpha^3+16 #leq 139_17976.jpg
Finished:  582
Filename:  371#alpha^4+477 #lt 939_8189.jpg
Finished:  583
Filename:  93+689 = 782_93277.jpg
Finished:  584
Filename:  496+702 = 1198_77477.jpg
Finished:  585
Filename:  966y^4-148 #leq 881_34951.jpg
Finished:  586
Filename:  764y^3-106 #lt 691_18141.jpg
Finished:  587
Filename:  717y^4+337 #neq 1105_34199.jpg
Finished:  588
Filename:  879#beta^2+684 #leq 1661_33757.jpg
Finished:  589
Filename:  148#alpha^2-979 #gt -848_21075.jpg
Finished:  590
Filename:  821#lambda^2+275 #lt 1105_48286.jpg
Finished:  591
Filename:  397y^2+489 #geq 845_60086.jpg
Finished:  592
Filename:  285#alpha^3+366 #gt 621_77121.jpg
Finished:  593
Filename:  20y^2-557 #geq -601_73678.jpg
Finished:  594
Filename:  787#beta^3-215 #leq 597_63489.jpg
Finished:  595
Filename:  242x^4+625 #gt 865_27772.jpg
Finished:  596
Filename:  7

...

Finished:  4423
Filename:  412#beta^2+907 #geq 1269_78520.jpg
Finished:  4424
Filename:  578y^2+406 #leq 1049_4426.jpg
Finished:  4425
Filename:  706#lambda^4-354 #gt 321_94836.jpg
Finished:  4426
Filename:  429y^2+2 #leq 441_58134.jpg
Finished:  4427
Filename:  695#alpha^4-723 #neq -2_95659.jpg
Finished:  4428
Filename:  308x^4-601 #lt -256_64595.jpg
Finished:  4429
Filename:  306y^3+727 #gt 950_27956.jpg
Finished:  4430
Filename:  377+128 = 505_52491.jpg
Finished:  4431
Filename:  331y^3-413 #geq -135_9267.jpg
Finished:  4432
Filename:  534x^3+749 #geq 1194_40025.jpg
Finished:  4433
Filename:  200#lambda^4-310 #gt -138_68296.jpg
Finished:  4434
Filename:  241y^2+3 #geq 237_27522.jpg
Finished:  4435
Filename:  352#beta^2-135 #leq 307_60851.jpg
Finished:  4436
Filename:  515y^4+126 #geq 584_75123.jpg
Finished:  4437
Filename:  616#alpha^3+509 #gt 1067_5771.jpg
Finished:  4438
Filename:  847x^4+176 #geq 957_10516.jpg
Finished:  4439
Filename:  108y^3-231 #geq -133_49669.jpg
Finished:  4

Finished:  4564
Filename:  140x^3+943 #neq 1094_14146.jpg
Finished:  4565
Filename:  618#lambda^4+290 #neq 1001_50855.jpg
Finished:  4566
Filename:  356x^2+865 #leq 1305_33909.jpg
Finished:  4567
Filename:  835y^4-9 #neq 885_49646.jpg
Finished:  4568
Filename:  140#beta^4+99 #neq 268_69658.jpg
Finished:  4569
Filename:  237x^3+625 #leq 934_52811.jpg
Finished:  4570
Filename:  876+203 = 1079_64254.jpg
Finished:  4571
Filename:  983+549 = 1532_98677.jpg
Finished:  4572
Filename:  811#alpha^2-359 #leq 467_21000.jpg
Finished:  4573
Filename:  350#beta^3-781 #geq -517_62018.jpg
Finished:  4574
Filename:  396#lambda^3-317 #lt 156_39736.jpg
Finished:  4575
Filename:  124x^4-865 #neq -716_35583.jpg
Finished:  4576
Filename:  458-911 = -453_88666.jpg
Finished:  4577
Filename:  160#lambda^2-285 #geq -209_38658.jpg
Finished:  4578
Filename:  375#lambda^4+104 #geq 420_54118.jpg
Finished:  4579
Filename:  600y^4-980 #neq -339_69986.jpg
Finished:  4580
Filename:  617+513 = 1130_83051.jpg
Finished:  

Finished:  4707
Filename:  650#lambda^3-1 #lt 664_8833.jpg
Finished:  4708
Filename:  428#lambda^2-722 #leq -218_83825.jpg
Finished:  4709
Filename:  52x^3-442 #gt -471_66723.jpg
Finished:  4710
Filename:  258#lambda^3+714 #neq 1003_65318.jpg
Finished:  4711
Filename:  152#alpha^2-448 #neq -289_74456.jpg
Finished:  4712
Filename:  188x^4-741 #geq -574_6008.jpg
Finished:  4713
Filename:  88y^4-86 #neq 5_9303.jpg
Finished:  4714
Filename:  954#alpha^2+500 #lt 1493_53872.jpg
Finished:  4715
Filename:  159y^4-259 #leq -14_92756.jpg
Finished:  4716
Filename:  992#lambda^2+115 #lt 1194_189.jpg
Finished:  4717
Filename:  451#beta^3+179 #leq 644_93988.jpg
Finished:  4718
Filename:  223#beta^4+194 #lt 484_20766.jpg
Finished:  4719
Filename:  419#lambda^3-147 #geq 176_94098.jpg
Finished:  4720
Filename:  78-26 = 52_79913.jpg
Finished:  4721
Filename:  379y^4+670 #gt 966_22836.jpg
Finished:  4722
Filename:  760#lambda^4-910 #neq -63_75742.jpg
Finished:  4723
Filename:  54+273 = 327_16891.jpg
Fini

Finished:  4849
Filename:  347-124 = 223_28002.jpg
Finished:  4850
Filename:  628+128 = 756_14991.jpg
Finished:  4851
Filename:  168#lambda^2-442 #geq -360_84930.jpg
Finished:  4852
Filename:  855y^3+591 #geq 1442_8026.jpg
Finished:  4853
Filename:  796y^4-134 #neq 746_60243.jpg
Finished:  4854
Filename:  298#alpha^3-852 #neq -514_59062.jpg
Finished:  4855
Filename:  612#alpha^3+799 #lt 1429_60628.jpg
Finished:  4856
Filename:  922#beta^4-21 #leq 995_83825.jpg
Finished:  4857
Filename:  884#beta^4-565 #geq 260_47620.jpg
Finished:  4858
Filename:  767y^2-126 #leq 740_10519.jpg
Finished:  4859
Filename:  341#alpha^4-160 #lt 256_70489.jpg
Finished:  4860
Filename:  709#beta^3+396 #lt 1155_95392.jpg
Finished:  4861
Filename:  515+97 = 612_37965.jpg
Finished:  4862
Filename:  979#alpha^3-603 #neq 452_85257.jpg
Finished:  4863
Filename:  577#beta^2+665 #leq 1251_12015.jpg
Finished:  4864
Filename:  235x^4+118 #leq 412_92419.jpg
Finished:  4865
Filename:  959#alpha^2-388 #gt 571_38375.jpg
Fin

Finished:  4991
Filename:  228#beta^2-261 #lt 66_12116.jpg
Finished:  4992
Filename:  364x^2-946 #neq -531_93266.jpg
Finished:  4993
Filename:  944#alpha^2-64 #neq 908_55780.jpg
Finished:  4994
Filename:  909#alpha^4+853 #gt 1681_32889.jpg
Finished:  4995
Filename:  375#beta^3+765 #gt 1099_14054.jpg
Finished:  4996
Filename:  202#beta^3+352 #neq 577_15967.jpg
Finished:  4997
Filename:  897-146 = 751_71627.jpg
Finished:  4998
Filename:  902x^4-996 #geq -103_75144.jpg
Finished:  4999


In [55]:
# fractions
list_digits = []
for i in range(10):
    list_digits.append(listdir(read_path+"/"+str(i)))
list_plus = listdir(read_path+"/+")
list_minus = listdir(read_path+"/-")
list_leq = listdir(read_path+"/leq")
list_geq = listdir(read_path+"/geq")
list_neq = listdir(read_path+"/neq")
list_equal = listdir(read_path+"/=")

list_alpha = listdir(read_path+"/alpha")
list_beta = listdir(read_path+"/beta")
list_lambda = listdir(read_path+"/lambda")
list_x = listdir(read_path+"/x")
list_y = listdir(read_path+"/y")

if train: 
    for i in range(10):
        list_digits[i] = list_digits[i][len(list_digits[i])//4:]
    list_plus = list_plus[len(list_plus)//4:]
    list_minus = list_minus[len(list_minus)//4:]
    list_leq = list_leq[len(list_leq)//4:]
    list_geq = list_geq[len(list_geq)//4:]
    list_neq = list_neq[len(list_neq)//4:]
    list_equal = list_equal[len(list_equal)//4:]
    list_alpha = list_alpha[len(list_alpha)//4:]
    list_beta = list_beta[len(list_beta)//4:]
    list_lambda = list_lambda[len(list_lambda)//4:]
    list_x = list_x[len(list_x)//4:]
    list_y = list_y[len(list_y)//4:]
else:
    for i in range(10):
        list_digits[i] = list_digits[i][:len(list_digits[i])//4]
    list_plus = list_plus[:len(list_plus)//4]
    list_minus = list_minus[:len(list_minus)//4]
    list_leq = list_leq[:len(list_leq)//4]
    list_geq = list_geq[:len(list_geq)//4]
    list_neq = list_neq[:len(list_neq)//4]
    list_equal = list_equal[:len(list_equal)//4]
    list_alpha = list_alpha[:len(list_alpha)//4]
    list_beta = list_beta[:len(list_beta)//4]
    list_lambda = list_lambda[:len(list_lambda)//4]
    list_x = list_x[:len(list_x)//4]
    list_y = list_y[:len(list_y)//4]

list_mid = [list_minus,list_plus]
list_end = [list_leq,list_geq,list_neq,list_equal]  
list_variables = [list_alpha,list_beta,list_lambda,list_x,list_y]
var_names = ["#alpha","#beta","#lambda","x","y"]

bounding_boxes = []
for i in range(20000):
    random_name = str(np.random.randint(1,99999))
    img = np.zeros((200,60*(4+4+4)))
    rand_num_1 = np.random.randint(1,1000)
    rand_num_1_1 = np.random.randint(1,1000)
    rand_num_2 = np.random.randint(1,1000)
    mid = np.random.randint(2)
    if mid:
        mid_str = "+"
        result = rand_num_1+rand_num_2
    else:
        mid_str = "-"
        result = rand_num_1-rand_num_2
    result_type = np.random.randint(4)
    if result_type == 0:
        end_str = " #leq "
        result += np.random.randint(100)
    elif result_type == 1:
        end_str = " #geq "
        result -= np.random.randint(100)
    elif result_type == 2:
        end_str = " #neq "
        result += 1+np.random.randint(100)
    else:
        end_str = " = "
    if end_str != " = ":
        var_type = np.random.randint(len(list_variables))
        var = var_names[var_type]
        exp = "^"+str(np.random.randint(2,5))
    else:
        var = ""
        exp = ""
    
    rand_num_1_str = str(rand_num_1)
    rand_num_1_1_str = str(rand_num_1_1)
    rand_num_2_str = str(rand_num_2)
    result_str = str(result)
    num_strs = [rand_num_1_str,rand_num_1_1_str,rand_num_2_str,result_str]
    filename = "#frac{"+rand_num_1_str+"}{"+rand_num_1_1_str+"}"+var+exp+mid_str+rand_num_2_str+end_str+result_str+"_"+random_name+".jpg"
    print("Filename: ", filename)
    bounding_box = [{'filename': filename}]
    padding = 5
    padding_den = 5
    start_padding = 5
    class_names = []
    for k in range(len(num_strs)):
        for j in range(len(num_strs[k])):   
            if j == 0 and num_strs[k][0] == '-':
                img, padding, new_bounding_box = add_symbol_to_image(img,'-',list_minus,padding,39,45)
                bounding_box.append(new_bounding_box)
                class_names.append('-')
            elif k == 0:
                digit = int(num_strs[k][j])
                img, padding, new_bounding_box = add_symbol_to_image(img,str(digit),list_digits[digit],
                                                                     padding,40,45,bnom=True)
                bounding_box.append(new_bounding_box)
                class_names.append(str(digit))
            elif k == 1:
                digit = int(num_strs[k][j])
                img, padding_den, new_bounding_box = add_symbol_to_image(img,str(digit),list_digits[digit],
                                                                     padding_den,40,45,bden=True)
                bounding_box.append(new_bounding_box)
                class_names.append(str(digit))
            else:               
                digit = int(num_strs[k][j])
                img, padding, new_bounding_box = add_symbol_to_image(img,str(digit),list_digits[digit],
                                                                     padding,55,60)
                bounding_box.append(new_bounding_box)
                class_names.append(str(digit))
        if k == 1:
            mpad = padding if padding > padding_den else padding_den
            img, padding, new_bounding_box = add_symbol_to_image(img,'-',list_minus,start_padding,39,45,
                                                                 width=mpad-start_padding)
            bounding_box.append(new_bounding_box)
            class_names.append('-')
        if k == 1:
            if var != "":
                var_str_crop = var.strip()
                var_str_crop = var_str_crop.replace("#","")
                img, padding, new_bounding_box = add_symbol_to_image(img,var_str_crop,list_variables[var_type],
                                                                     padding,40,45,bsmall=True)
                bounding_box.append(new_bounding_box)
                class_names.append(var_str_crop)
                
                pdigit = int(exp[1:])
                img, padding, new_bounding_box = add_symbol_to_image(img,str(pdigit),list_digits[pdigit],
                                                                     padding,35,40,bpower=True)
                bounding_box.append(new_bounding_box)
                class_names.append(exp[1:])
                
            img, padding, new_bounding_box = add_symbol_to_image(img,mid_str,list_mid[mid],
                                                                     padding,39,54)
            bounding_box.append(new_bounding_box)
            class_names.append(mid_str)
        elif k == 2:
            end_str_crop = end_str.strip()
            end_str_crop = end_str_crop.replace("#","")
            img, padding, new_bounding_box = add_symbol_to_image(img,end_str_crop,list_end[result_type],
                                                                     padding,39,54)
            bounding_box.append(new_bounding_box)
            class_names.append(end_str_crop)
            
    bounding_boxes.append(bounding_box)
    img = invert(img)+254
#     plt.figure(figsize=(20,10))
#     plt.imshow(img, cmap="gray")
#     plt.show()
    """
    for bb,cname in zip(bounding_box[1:],class_names):
        xmin, xmax = bb['xmin'], bb['xmax']
        ymin, ymax = bb['ymin'], bb['ymax']
    
        normed = normalize_single(img[ymin:ymax+1,xmin:xmax+1])
        r = np.random.randint(9999)
        io.imsave(write_single_path+"/"+cname+"/"+cname+"_"+str(r)+".jpg", normed)
    """
    if not path.exists(formula_path):
        mkdir(formula_path)
    io.imsave(formula_path+"/"+filename, img/255)
    print("Finished: ", i)

Filename:  #frac{82}{90}#lambda^2+568 #leq 678_61016.jpg
Finished:  0
Filename:  #frac{657}{780}+303 = 960_81342.jpg
Finished:  1
Filename:  #frac{648}{29}-828 = -180_2024.jpg
Finished:  2
Filename:  #frac{566}{240}y^2-425 #neq 196_13660.jpg
Finished:  3
Filename:  #frac{528}{460}+623 = 1151_37570.jpg
Finished:  4
Filename:  #frac{367}{170}x^3-686 #geq -348_49693.jpg
Finished:  5
Filename:  #frac{796}{356}#lambda^2-862 #neq -64_50659.jpg
Finished:  6
Filename:  #frac{454}{596}x^4+75 #neq 621_80619.jpg
Finished:  7
Filename:  #frac{635}{60}-350 = 285_21935.jpg
Finished:  8
Filename:  #frac{882}{933}-867 = 15_39889.jpg
Finished:  9
Filename:  #frac{292}{798}#beta^4+429 #neq 759_16644.jpg
Finished:  10
Filename:  #frac{46}{127}#beta^4+849 #geq 802_6313.jpg
Finished:  11
Filename:  #frac{997}{280}#alpha^2-157 #neq 867_97207.jpg
Finished:  12
Filename:  #frac{778}{541}-81 = 697_24280.jpg
Finished:  13
Filename:  #frac{380}{499}y^3+459 #neq 899_49849.jpg
Finished:  14
Filename:  #frac{806}{9

Finished:  124
Filename:  #frac{717}{697}+901 = 1618_49301.jpg
Finished:  125
Filename:  #frac{694}{286}x^2+435 #neq 1200_14560.jpg
Finished:  126
Filename:  #frac{966}{699}#lambda^3+308 #leq 1297_86503.jpg
Finished:  127
Filename:  #frac{561}{847}+948 = 1509_23532.jpg
Finished:  128
Filename:  #frac{101}{739}#lambda^2-510 #leq -355_36328.jpg
Finished:  129
Filename:  #frac{965}{613}x^4+498 #neq 1497_70401.jpg
Finished:  130
Filename:  #frac{740}{984}+55 = 795_50263.jpg
Finished:  131
Filename:  #frac{295}{210}x^4+428 #geq 649_99117.jpg
Finished:  132
Filename:  #frac{491}{828}#beta^3-426 #geq 27_815.jpg
Finished:  133
Filename:  #frac{744}{526}#beta^4+99 #leq 910_19707.jpg
Finished:  134
Filename:  #frac{348}{308}#beta^3+476 #leq 914_53759.jpg
Finished:  135
Filename:  #frac{814}{563}y^4-944 #geq -158_78345.jpg
Finished:  136
Filename:  #frac{77}{603}y^3+312 #neq 481_45317.jpg
Finished:  137
Filename:  #frac{726}{852}#alpha^3-97 #leq 700_37459.jpg
Finished:  138
Filename:  #frac{495}{

Finished:  244
Filename:  #frac{543}{809}+874 = 1417_83556.jpg
Finished:  245
Filename:  #frac{718}{800}#alpha^2-805 #neq 13_57894.jpg
Finished:  246
Filename:  #frac{370}{735}y^4-23 #leq 412_42793.jpg
Finished:  247
Filename:  #frac{382}{11}#alpha^2-984 #neq -511_22875.jpg
Finished:  248
Filename:  #frac{432}{181}#beta^3+344 #neq 782_19474.jpg
Finished:  249
Filename:  #frac{137}{266}+103 = 240_25819.jpg
Finished:  250
Filename:  #frac{360}{984}#lambda^4+374 #neq 764_53253.jpg
Finished:  251
Filename:  #frac{146}{687}y^2-61 #geq -6_88751.jpg
Finished:  252
Filename:  #frac{395}{156}y^3-162 #neq 313_26299.jpg
Finished:  253
Filename:  #frac{818}{562}#beta^3+866 #leq 1711_48835.jpg
Finished:  254
Filename:  #frac{766}{298}#alpha^3-559 #neq 307_87149.jpg
Finished:  255
Filename:  #frac{92}{572}+433 = 525_83877.jpg
Finished:  256
Filename:  #frac{749}{693}#beta^3-413 #neq 361_79556.jpg
Finished:  257
Filename:  #frac{362}{707}#lambda^4+908 #geq 1260_15440.jpg
Finished:  258
Filename:  #fr

Finished:  365
Filename:  #frac{134}{567}y^2+845 #neq 1031_95247.jpg
Finished:  366
Filename:  #frac{961}{149}#lambda^3-541 #geq 407_71428.jpg
Finished:  367
Filename:  #frac{665}{244}#alpha^2-333 #geq 278_72545.jpg
Finished:  368
Filename:  #frac{355}{683}#alpha^2+182 #leq 551_61534.jpg
Finished:  369
Filename:  #frac{276}{847}+595 = 871_51987.jpg
Finished:  370
Filename:  #frac{267}{66}+188 = 455_85425.jpg
Finished:  371
Filename:  #frac{684}{378}+103 = 787_69393.jpg
Finished:  372
Filename:  #frac{156}{180}#beta^4-47 #geq 105_89274.jpg
Finished:  373
Filename:  #frac{441}{817}-553 = -112_29938.jpg
Finished:  374
Filename:  #frac{634}{336}#lambda^4-154 #neq 534_84154.jpg
Finished:  375
Filename:  #frac{837}{546}-203 = 634_57246.jpg
Finished:  376
Filename:  #frac{480}{199}x^4-121 #leq 399_99460.jpg
Finished:  377
Filename:  #frac{363}{481}+173 = 536_64454.jpg
Finished:  378
Filename:  #frac{602}{606}#beta^2+678 #geq 1260_56021.jpg
Finished:  379
Filename:  #frac{648}{814}+436 = 1084_

Finished:  485
Filename:  #frac{210}{435}#lambda^2+309 #geq 480_99536.jpg
Finished:  486
Filename:  #frac{352}{525}+958 = 1310_86119.jpg
Finished:  487
Filename:  #frac{252}{387}#alpha^2+98 #neq 385_50724.jpg
Finished:  488
Filename:  #frac{717}{604}-766 = -49_37195.jpg
Finished:  489
Filename:  #frac{723}{507}#beta^2-939 #geq -235_14158.jpg
Finished:  490
Filename:  #frac{426}{603}-623 = -197_59442.jpg
Finished:  491
Filename:  #frac{119}{987}x^3+995 #geq 1089_76309.jpg
Finished:  492
Filename:  #frac{443}{133}x^2+177 #neq 640_65238.jpg
Finished:  493
Filename:  #frac{545}{132}x^4+215 #geq 759_24213.jpg
Finished:  494
Filename:  #frac{820}{686}+582 = 1402_57116.jpg
Finished:  495
Filename:  #frac{676}{177}+559 = 1235_6455.jpg
Finished:  496
Filename:  #frac{381}{719}+581 = 962_17096.jpg
Finished:  497
Filename:  #frac{175}{482}#lambda^4-376 #leq -164_24463.jpg
Finished:  498
Filename:  #frac{751}{327}y^2-171 #neq 595_32833.jpg
Finished:  499
Filename:  #frac{671}{763}#alpha^3+562 #leq

...

Finished:  19428
Filename:  #frac{89}{898}+428 = 517_99258.jpg
Finished:  19429
Filename:  #frac{519}{606}x^3+348 #leq 937_88918.jpg
Finished:  19430
Filename:  #frac{74}{129}#beta^3-202 #neq -48_25491.jpg
Finished:  19431
Filename:  #frac{499}{657}-617 = -118_34196.jpg
Finished:  19432
Filename:  #frac{244}{441}#beta^4-345 #geq -126_32323.jpg
Finished:  19433
Filename:  #frac{819}{354}#alpha^3+386 #neq 1254_89758.jpg
Finished:  19434
Filename:  #frac{700}{840}#lambda^3-256 #leq 460_43131.jpg
Finished:  19435
Filename:  #frac{803}{699}#beta^4+337 #leq 1149_52001.jpg
Finished:  19436
Filename:  #frac{270}{595}y^4-598 #leq -325_95125.jpg
Finished:  19437
Filename:  #frac{651}{415}#lambda^3-758 #geq -170_40312.jpg
Finished:  19438
Filename:  #frac{309}{364}y^3-344 #leq 51_36069.jpg
Finished:  19439
Filename:  #frac{185}{207}y^4-401 #leq -170_87379.jpg
Finished:  19440
Filename:  #frac{302}{667}y^4-660 #neq -294_52011.jpg
Finished:  19441
Filename:  #frac{3}{448}y^4+930 #leq 1012_86041.jpg

Finished:  19546
Filename:  #frac{434}{165}x^3+752 #leq 1200_70193.jpg
Finished:  19547
Filename:  #frac{192}{276}#beta^4+119 #neq 312_62574.jpg
Finished:  19548
Filename:  #frac{435}{618}x^3+779 #geq 1154_86999.jpg
Finished:  19549
Filename:  #frac{87}{791}x^3-869 #leq -758_43118.jpg
Finished:  19550
Filename:  #frac{434}{860}#alpha^4-892 #leq -359_67648.jpg
Finished:  19551
Filename:  #frac{783}{245}-546 = 237_89936.jpg
Finished:  19552
Filename:  #frac{787}{708}#lambda^4+562 #leq 1357_82507.jpg
Finished:  19553
Filename:  #frac{242}{5}#lambda^3+332 #geq 492_75658.jpg
Finished:  19554
Filename:  #frac{76}{163}#lambda^2+614 #geq 609_42757.jpg
Finished:  19555
Filename:  #frac{563}{674}-493 = 70_14684.jpg
Finished:  19556
Filename:  #frac{735}{119}#beta^3-171 #geq 500_75026.jpg
Finished:  19557
Filename:  #frac{713}{34}y^3-567 #leq 203_37111.jpg
Finished:  19558
Filename:  #frac{421}{927}#alpha^4-38 #leq 465_13167.jpg
Finished:  19559
Filename:  #frac{709}{862}#lambda^2-684 #geq 11_111

Finished:  19663
Filename:  #frac{798}{844}-481 = 317_13157.jpg
Finished:  19664
Filename:  #frac{366}{428}#lambda^3-922 #leq -536_47549.jpg
Finished:  19665
Filename:  #frac{676}{103}#beta^2-760 #leq -40_24367.jpg
Finished:  19666
Filename:  #frac{800}{599}y^3+465 #geq 1248_57487.jpg
Finished:  19667
Filename:  #frac{16}{945}#alpha^2-473 #neq -432_36943.jpg
Finished:  19668
Filename:  #frac{729}{594}#lambda^2-680 #neq 97_27791.jpg
Finished:  19669
Filename:  #frac{176}{460}#beta^4+879 #neq 1105_37209.jpg
Finished:  19670
Filename:  #frac{949}{370}x^2+141 #leq 1113_79711.jpg
Finished:  19671
Filename:  #frac{183}{739}x^4-703 #neq -491_25057.jpg
Finished:  19672
Filename:  #frac{727}{148}+815 = 1542_34439.jpg
Finished:  19673
Filename:  #frac{561}{30}#lambda^2-184 #neq 432_35489.jpg
Finished:  19674
Filename:  #frac{921}{382}x^2-284 #neq 647_57200.jpg
Finished:  19675
Filename:  #frac{190}{970}+838 = 1028_61025.jpg
Finished:  19676
Filename:  #frac{869}{950}y^3+179 #geq 967_64080.jpg
Fi

Finished:  19787
Filename:  #frac{396}{819}#beta^3+946 #neq 1388_94057.jpg
Finished:  19788
Filename:  #frac{92}{707}#alpha^3+624 #geq 620_71713.jpg
Finished:  19789
Filename:  #frac{550}{545}+585 = 1135_9170.jpg
Finished:  19790
Filename:  #frac{975}{270}-666 = 309_33635.jpg
Finished:  19791
Filename:  #frac{275}{983}#lambda^3+72 #leq 438_76958.jpg
Finished:  19792
Filename:  #frac{143}{333}#lambda^4-838 #leq -674_11694.jpg
Finished:  19793
Filename:  #frac{949}{330}x^2-201 #geq 711_93731.jpg
Finished:  19794
Filename:  #frac{882}{418}+173 = 1055_25678.jpg
Finished:  19795
Filename:  #frac{631}{813}y^2-429 #geq 124_7823.jpg
Finished:  19796
Filename:  #frac{75}{844}y^2+860 #neq 950_4979.jpg
Finished:  19797
Filename:  #frac{159}{890}#alpha^2-491 #leq -276_96786.jpg
Finished:  19798
Filename:  #frac{104}{431}x^2-223 #geq -196_31122.jpg
Finished:  19799
Filename:  #frac{100}{311}+253 = 353_47818.jpg
Finished:  19800
Filename:  #frac{509}{586}+525 = 1034_95714.jpg
Finished:  19801
Filena

Finished:  19904
Filename:  #frac{283}{619}#beta^4+983 #geq 1191_71291.jpg
Finished:  19905
Filename:  #frac{152}{800}x^3+424 #geq 539_3606.jpg
Finished:  19906
Filename:  #frac{219}{882}#lambda^3+160 #geq 314_46815.jpg
Finished:  19907
Filename:  #frac{951}{669}+520 = 1471_8544.jpg
Finished:  19908
Filename:  #frac{431}{164}y^3+58 #geq 453_86919.jpg
Finished:  19909
Filename:  #frac{350}{200}-102 = 248_6317.jpg
Finished:  19910
Filename:  #frac{301}{971}#lambda^4-676 #geq -455_34463.jpg
Finished:  19911
Filename:  #frac{838}{154}y^3-519 #neq 355_109.jpg
Finished:  19912
Filename:  #frac{381}{988}#alpha^4-453 #neq 19_18527.jpg
Finished:  19913
Filename:  #frac{484}{940}#alpha^3-581 #neq -31_12646.jpg
Finished:  19914
Filename:  #frac{847}{304}#beta^4+227 #leq 1127_63617.jpg
Finished:  19915
Filename:  #frac{17}{555}y^2+685 #neq 794_64367.jpg
Finished:  19916
Filename:  #frac{726}{378}#alpha^4-790 #geq -83_74270.jpg
Finished:  19917
Filename:  #frac{688}{69}y^2-729 #leq 12_69196.jpg
Fin